In [1]:
from soil.agents import FSM, state, default_state
from soil import Environment
from random import random, shuffle, sample
from itertools import islice
import logging
from enum import Enum
from random import randint
from soil import simulation
from soil import *
import soil
import networkx as nx
import logging
logging.basicConfig()


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:

class Genders(Enum):
    male = 'male'
    female = 'female'

class Venues(Enum):
    pub = 'pub'
    disco = 'disco'
    street = 'street'


class CityPubs(Environment):

    """
    The environment is key in a simulation. It contains the network topology,
    a reference to network and environment agents, as well as the environment
    params, which are used as shared state between agents.
    The environment parameters and the state of every agent can be accessed
    both by using the environment as a dictionary or with the environment's 
    :meth:`soil.environment.Environment.get` method.
    'name': 'The awesome pub #{}'.format(i),
                'open': True,
                'capacity': pub_capacity ,
                'occupancy': 0,
                'price':randint(4, 6),
                'type': Venues.pub.value,
    """

    '''Environment with Pubs'''
    level = logging.INFO

    # ----------------------------------INICIALIZADOR----------------------------------------------------------------------------
    """def __init__(self, *args, number_of_pubs=3, number_of_discos=3, number_of_street=3, pub_capacity=10,
                 disco_capacity = 20, street_capacity = 30, **kwargs):
                    super(CityPubs, self).__init__(*args, **kwargs)  #Para la clase superior. Constructor super.
                    pubs = {}
                    for i in range(number_of_pubs):
                        newpub = {
                            'name': 'The awesome pub #{}'.format(i),
                            'open': True,
                            'capacity': pub_capacity ,
                            'occupancy': 0,
                            'price':randint(4, 6),
                            'type': Venues.pub.value,
                        }
                        pubs[newpub['name']] = newpub
                    for i in range(number_of_discos):
                        newpub = {
                            'name': 'The awesome disco #{}'.format(i),
                            'open': True,
                            'capacity': disco_capacity ,
                            'occupancy': 0,
                            'price':randint(7, 9),
                            'type': Venues.disco.value,
                        }
                        pubs[newpub['name']] = newpub
                    for i in range(number_of_street):
                        newpub = {
                            'name': 'The awesome street #{}'.format(i),
                            'open': True,
                            'capacity': pub_capacity ,
                            'occupancy': 0,
                            'price':randint(2,4),
                            'type': Venues.street.value,
                        }
                        pubs[newpub['name']] = newpub
            
                        
                    self['pubs'] = pubs"""

                    
    def __init__(self, *args, pubs=None, **kwargs):
        super(CityPubs, self).__init__(*args, **kwargs)  #Para la clase superior. Constructor super.

            
        self['pubs'] = pubs or {}

        """ >>> tel = {'jack': 4098, 'sape': 4139}
            >>> tel['guido'] = 4127    
            >>> tel
            {'sape': 4139, 'guido': 4127, 'jack': 4098}"""

        #Hacemos un bucle y llenamos la variable pubs con los bares que va a haber: queda así:
            #{'The awesome pub #2': {'name': 'The awesome pub #2', 'open': True, 'capacity': 10, 'occupancy': 0}, 
            # 'The awesome pub #1': {'name': 'The awesome pub #1', 'open': True, 'capacity': 10, 'occupancy': 0}, 
            # 'The awesome pub #0': {'name': 'The awesome pub #0', 'open': True, 'capacity': 10, 'occupancy': 0}}


    #----------------------------------------MÉTODOS-------------------------------------------------------------------------------------
   

    def return_occupancy (self,pub_name):
        pub = self['pubs'][pub_name]
        return pub['occupancy']

    def return_price(self,pub_name):
        pub = self['pubs'][pub_name]
        return pub['price']

    def return_name(self,pub_name):
        pub = self['pubs'][pub_name]
        return pub['name']

    def return_type(self,pub_name):
        pub = self['pubs'][pub_name]
        return pub['type']




    def enter(self, pub_name, *nodes):

        '''Agents will try to enter. The pub checks if it is possible'''
        #A este método se le pasa el id del pub al que quieren entrar, y el grupo de amigos.
        #Hace comprobaciones con la capacidad y viendo si está abierto. Devuelve True si se puede entrar.

        try:
            pub = self['pubs'][pub_name]
        except KeyError:
            raise ValueError('Pub {} is not available'.format(pub_name))
        if not pub['open'] or (pub['capacity'] < (len(nodes) + pub['occupancy'])):

            return False
        pub['occupancy'] += len(nodes)
        for node in nodes:
            node['pub'] = pub_name
        return True


    #Devuelve una lista de pubs en los que se puede entrar. Lo hace con yield: se genera un objeto en vez de 
    # una lista. Cuando se llama al método no se genera la lista, se genera un objeto. SOlo cuando intentemos recorrerlo
    # se generará una lista que además solo puede recorrerse una vez --> https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do
    """def available_pubs(self):
                    
                    for pub in self['pubs'].values():
                        if pub['open'] and (pub['occupancy'] < pub['capacity']):
                            yield pub['name']"""

    def available_pubs(self):
        available_venues = []
        for pub in self['pubs'].values():
            if pub['open'] and (pub['occupancy'] < pub['capacity']):
                available_venues.append(pub['name'])

        shuffle(available_venues)
        return available_venues 


    #Un grupo se va de un pub. La sintaxis "del" es como decir que te vacíe esa variable
    """def exit(self, pub_id, *node_ids):
                    '''Agents will notify the pub they want to leave'''
                    try:
                        pub = self['pubs'][pub_id]
                    except KeyError:
                        raise ValueError('Pub {} is not available'.format(pub_id))
                    for node_id in node_ids:
                        node = self.get_agent(node_id)
                        if pub_id == node['pub']:
                            del node['pub']
                            pub['occupancy'] -= 1
            
                def exit(self, pub_id, *nodes):
                    '''Agents will notify the pub they want to leave'''
                    try:
                        pub = self['pubs'][pub_id]
                    except KeyError:
                        raise ValueError('Pub {} is not available'.format(pub_id))
                    for node in nodes:
                     
                        if pub_id == node['pub']:
                            del node['pub']
                            pub['occupancy'] -= 1
            """
    def exit(self, pub_name, *nodes):

        try:
            pub = self['pubs'][pub_name]
        except KeyError:
            raise ValueError('Pub {} is not available'.format(pub_name))
        
        pub['occupancy'] -= len(nodes)



In [3]:
class Patron(FSM):
    '''Agent that looks for friends to drink with. It will do three things:
        1) Look for other patrons to drink with
        2) Look for a bar where the agent and other agents in the same group can get in.
        3) While in the bar, patrons only drink, until they get drunk and taken home.
    '''
    level = logging.INFO

    defaults = {
        'pub': None,
        'drunk': False,
        'pints': 0,
        'max_pints': 5,
        'in_a_group':False,
        'gender': Genders.male.value,
        'money':20,
        'is_leader': False,
        ##'interval'
    }

    @default_state
    @state
    def looking_for_friends(self):
        '''Look for friends to drink with'''

        if(self['in_a_group'] == False):
            self.info('I am looking for friends')
            self['is_leader'] = True
            available_friends = list(self.get_agents(drunk=False,
                                                     pub=None,
                                                     in_a_group=False))
                                                     
            if not available_friends or len(available_friends)==1:
                self.info('Life sucks and I\'m alone!')
                return self.at_home
            befriended = self.try_friends(available_friends)
            if befriended:
                
                return self.looking_for_pub#, self.env.timeout(3)
        else:
            self.info('{} has a group already' .format(self.id))
            return self.looking_for_pub

    @state
    def looking_for_pub(self):
        '''Look for a pub that accepts me and my friends'''
        if self['pub'] != None:
            return self.sober_in_pub
        self.debug('I am looking for a pub')
        group = list(self.get_neighboring_agents())


        available_pubs = self.env.available_pubs()

        for pub in available_pubs:


            self.debug('We\'re trying to get into {}: total: {}'.format(pub, len(group)))
            if self.env.enter(pub, self, *group):
                self.info('We\'re all {} getting in {}!'.format(len(group)+1, pub))
                capacity = self.env.return_occupancy(pub)
                self.info('{} now has {} people inside'.format(pub,capacity))
                return self.sober_in_pub
            else:
                self.info("We can\'t go inside {}".format(pub))

    @state
    def sober_in_pub(self):
        """Manipulamos prob_change_bar dependiendo de donde estén
                                Bar-->Disco probable
                                Bar-->Bar   bastante probable
                                Bar-->street probable
                                Street-->Bar probable
                                Street-->Disco probable
                                Street-->Street poco probable
                                Disco --> lo que sea poco probable(MIrar zonas como Nuit que hay varias discos)
                        
                                
                                Meter house para gente que copea en casa y luego va a discoteca?
                        
                        
                                En un futuro dependerá de la hora también"""

        type = self.env.return_type(self['pub'])
        self.info(type)
        
        if(type=="disco"):
            self['prob_change_bar'] = 0.01

        else:
            #Street
            self['prob_change_bar'] = 0.3


        if self['is_leader'] and (self['prob_change_bar']>random()):
            self.change_bar()



        '''Drink up.'''
        self.drink()
        if self['pints'] > self['max_pints']:
            return self.drunk_in_pub

    @state
    def drunk_in_pub(self):
        '''I'm out. Take me home!'''
        self.info('I\'m so drunk. Take me home!')
        self['drunk'] = True
        pass  # out drunk

    @state
    def at_home(self):
        '''The end'''
        self.debug('Life sucks. I\'m home!')
    
    def change_bar(self):
        self.info('This member is going to change pub: {}'.format(self.id))

        current_pub = self['pub']

        group = list(self.get_neighboring_agents())

        available_pubs = self.env.available_pubs()

        for pub in available_pubs:
            if self.env.return_name(pub) != current_pub:
                self.debug('We\'re trying to get into {}: total: {}'.format(pub, len(group)))
                if self.env.enter(pub, self, *group):
                    self.env.exit(current_pub,self, *group)
                    self.info('We\'re all {} changing to {}!'.format(len(group)+1, pub))
                    capacity = self.env.return_occupancy(pub)
                    self.info('{} now has {} people inside'.format(pub,capacity))
                    
                    return
                    
                else:
                    self.info("We can\'t go inside {}".format(pub))
        

                    #COMPLETAR#-----------


    
    def drink(self):
        price = self.env.return_price(self['pub'])
        if(self['prob_drink']>random() and price<self['money']):
            self['pints'] += 1
            self['money'] -= price
            self.debug('Cheers to that')
            
            self.debug('The price is {} € at {}'.format(price,self['pub']))

    def kick_out(self):
        self.set_state(self.at_home)

    def befriend(self, other_agent):
        '''
        Try to become friends with another agent. The chances of
        success depend on both agents' openness.
        '''
       
        self.env.add_edge(self, other_agent)
        self.info('Made some friend, agent {}'.format(other_agent.id))
        return True
        

    def try_friends(self, others):
        ''' Look for random agents around me and try to befriend them'''
        befriended = False
        k = randint(4, 6)
        shuffle(others)
        for friend in islice(others, k):  # random.choice >= 3.7
            if friend == self:
                continue
            if friend.befriend(self):
                self.befriend(friend)
                self.info('Hooray! new friend: {}'.format(friend.id))
                befriended = True
            else:
                self.info('{} does not want to be friends'.format(friend.id))

        self['in_a_group'] = True
        neighbors_leader = list(self.get_neighboring_agents())

        #print(*neighbors_leader)
        
        for people in neighbors_leader:
            people['in_a_group'] = True
            for i in neighbors_leader:
                if (people!=i):
                    people.befriend(i)
        
        return befriended


In [4]:
class Police(FSM):
    '''Simple agent to take drunk people out of pubs.'''
    level = logging.INFO

    @default_state
    @state
    def patrol(self):
        drunksters = list(self.get_agents(drunk=True,
                                          state_id=Patron.drunk_in_pub.id))
        for drunk in drunksters:
            self.info('Kicking out the trash: {}'.format(drunk.id))
            drunk.kick_out()
        else:
            self.info('No trash to take out. Too bad.')

In [5]:
simulation.run_from_config('pubcrawl.yml',
                               
                               dump=None,
                               )



INFO:soil:Using config(s): Malasaña_Heavy
INFO:soil:Dumping results to soil_output/Malasaña_Heavy : False
INFO:soil:Starting simulation Malasaña_Heavy at 14:59:26.
INFO:soil:Starting Simulation Malasaña_Heavy trial 0 at 14:59:28.
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    0:	No trash to take out. Too bad.
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    0:	I am looking for friends
INFO:Malasaña_Heavy_trial_0-Agent-14:	@    0:	Made some friend, agent 0
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    0:	Made some friend, agent 14
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    0:	Hooray! new friend: 14
INFO:Malasaña_Heavy_trial_0-Agent-81:	@    0:	Made some friend, agent 0
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    0:	Made some friend, agent 81
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    0:	Hooray! new friend: 81
INFO:Malasaña_Heavy_trial_0-Agent-68:	@    0:	Made some friend, agent 0
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    0:	Made some friend, agent 68
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    0:	Hooray

INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	I am looking for friends
INFO:Malasaña_Heavy_trial_0-Agent-85:	@    0:	Made some friend, agent 3
INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	Made some friend, agent 85
INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	Hooray! new friend: 85
INFO:Malasaña_Heavy_trial_0-Agent-33:	@    0:	Made some friend, agent 3
INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	Made some friend, agent 33
INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	Hooray! new friend: 33
INFO:Malasaña_Heavy_trial_0-Agent-79:	@    0:	Made some friend, agent 3
INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	Made some friend, agent 79
INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	Hooray! new friend: 79
INFO:Malasaña_Heavy_trial_0-Agent-29:	@    0:	Made some friend, agent 3
INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	Made some friend, agent 29
INFO:Malasaña_Heavy_trial_0-Agent-3:	@    0:	Hooray! new friend: 29
INFO:Malasaña_Heavy_trial_0-Agent-88:	@    0:	Made some friend, agent 3
INFO:Malas

INFO:Malasaña_Heavy_trial_0-Agent-45:	@    0:	Made some friend, agent 38
INFO:Malasaña_Heavy_trial_0-Agent-45:	@    0:	Made some friend, agent 30
INFO:Malasaña_Heavy_trial_0-Agent-30:	@    0:	Made some friend, agent 9
INFO:Malasaña_Heavy_trial_0-Agent-30:	@    0:	Made some friend, agent 34
INFO:Malasaña_Heavy_trial_0-Agent-30:	@    0:	Made some friend, agent 38
INFO:Malasaña_Heavy_trial_0-Agent-30:	@    0:	Made some friend, agent 45
INFO:Malasaña_Heavy_trial_0-Agent-6:	@    0:	6 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-7:	@    0:	I am looking for friends
INFO:Malasaña_Heavy_trial_0-Agent-58:	@    0:	Made some friend, agent 7
INFO:Malasaña_Heavy_trial_0-Agent-7:	@    0:	Made some friend, agent 58
INFO:Malasaña_Heavy_trial_0-Agent-7:	@    0:	Hooray! new friend: 58
INFO:Malasaña_Heavy_trial_0-Agent-44:	@    0:	Made some friend, agent 7
INFO:Malasaña_Heavy_trial_0-Agent-7:	@    0:	Made some friend, agent 44
INFO:Malasaña_Heavy_trial_0-Agent-7:	@    0:	Hooray! new friend: 44
IN

INFO:Malasaña_Heavy_trial_0-Agent-49:	@    0:	Made some friend, agent 26
INFO:Malasaña_Heavy_trial_0-Agent-49:	@    0:	Made some friend, agent 12
INFO:Malasaña_Heavy_trial_0-Agent-49:	@    0:	Made some friend, agent 94
INFO:Malasaña_Heavy_trial_0-Agent-26:	@    0:	Made some friend, agent 97
INFO:Malasaña_Heavy_trial_0-Agent-26:	@    0:	Made some friend, agent 98
INFO:Malasaña_Heavy_trial_0-Agent-26:	@    0:	Made some friend, agent 49
INFO:Malasaña_Heavy_trial_0-Agent-26:	@    0:	Made some friend, agent 12
INFO:Malasaña_Heavy_trial_0-Agent-26:	@    0:	Made some friend, agent 94
INFO:Malasaña_Heavy_trial_0-Agent-12:	@    0:	Made some friend, agent 97
INFO:Malasaña_Heavy_trial_0-Agent-12:	@    0:	Made some friend, agent 98
INFO:Malasaña_Heavy_trial_0-Agent-12:	@    0:	Made some friend, agent 49
INFO:Malasaña_Heavy_trial_0-Agent-12:	@    0:	Made some friend, agent 26
INFO:Malasaña_Heavy_trial_0-Agent-12:	@    0:	Made some friend, agent 94
INFO:Malasaña_Heavy_trial_0-Agent-94:	@    0:	Made 

INFO:Malasaña_Heavy_trial_0-Agent-20:	@    0:	Made some friend, agent 28
INFO:Malasaña_Heavy_trial_0-Agent-20:	@    0:	Hooray! new friend: 28
INFO:Malasaña_Heavy_trial_0-Agent-37:	@    0:	Made some friend, agent 20
INFO:Malasaña_Heavy_trial_0-Agent-20:	@    0:	Made some friend, agent 37
INFO:Malasaña_Heavy_trial_0-Agent-20:	@    0:	Hooray! new friend: 37
INFO:Malasaña_Heavy_trial_0-Agent-69:	@    0:	Made some friend, agent 39
INFO:Malasaña_Heavy_trial_0-Agent-69:	@    0:	Made some friend, agent 28
INFO:Malasaña_Heavy_trial_0-Agent-69:	@    0:	Made some friend, agent 61
INFO:Malasaña_Heavy_trial_0-Agent-69:	@    0:	Made some friend, agent 62
INFO:Malasaña_Heavy_trial_0-Agent-69:	@    0:	Made some friend, agent 37
INFO:Malasaña_Heavy_trial_0-Agent-39:	@    0:	Made some friend, agent 69
INFO:Malasaña_Heavy_trial_0-Agent-39:	@    0:	Made some friend, agent 28
INFO:Malasaña_Heavy_trial_0-Agent-39:	@    0:	Made some friend, agent 61
INFO:Malasaña_Heavy_trial_0-Agent-39:	@    0:	Made some fri

INFO:Malasaña_Heavy_trial_0-Agent-60:	@    0:	Made some friend, agent 84
INFO:Malasaña_Heavy_trial_0-Agent-60:	@    0:	Made some friend, agent 54
INFO:Malasaña_Heavy_trial_0-Agent-60:	@    0:	Made some friend, agent 57
INFO:Malasaña_Heavy_trial_0-Agent-60:	@    0:	Made some friend, agent 41
INFO:Malasaña_Heavy_trial_0-Agent-60:	@    0:	Made some friend, agent 75
INFO:Malasaña_Heavy_trial_0-Agent-33:	@    0:	33 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-34:	@    0:	34 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-35:	@    0:	35 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-36:	@    0:	36 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-37:	@    0:	37 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-38:	@    0:	38 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-39:	@    0:	39 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-40:	@    0:	I am looking for friends
INFO:Malasaña_Heavy_trial_0-Agent-66:	@    0:	Made some friend, agent 40
INFO:Mal

INFO:Malasaña_Heavy_trial_0-Agent-94:	@    0:	94 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-95:	@    0:	95 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-96:	@    0:	96 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-97:	@    0:	97 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-98:	@    0:	98 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-99:	@    0:	99 has a group already
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    1:	No trash to take out. Too bad.
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    1:	We're all 7 getting in Pacha!
INFO:Malasaña_Heavy_trial_0-Agent-0:	@    1:	Pacha now has 7 people inside
INFO:Malasaña_Heavy_trial_0-Agent-1:	@    1:	We're all 5 getting in Space Monkey!
INFO:Malasaña_Heavy_trial_0-Agent-1:	@    1:	Space Monkey now has 5 people inside
INFO:Malasaña_Heavy_trial_0-Agent-2:	@    1:	We're all 6 getting in Via Lactea!
INFO:Malasaña_Heavy_trial_0-Agent-2:	@    1:	Via Lactea now has 6 people inside
INFO:Malasaña_Heavy_trial_0-Age

INFO:Malasaña_Heavy_trial_0-Agent-92:	@    8:	I'm so drunk. Take me home!
INFO:Malasaña_Heavy_trial_0-Agent-98:	@    8:	I'm so drunk. Take me home!
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 3
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 7
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 11
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 13
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 23
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 26
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 31
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 32
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 36
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 44
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@    9:	Kicking out the trash: 50
INFO:Malasaña_Heav

INFO:Malasaña_Heavy_trial_0-Agent-1:	@   12:	This member is going to change pub: 1
INFO:Malasaña_Heavy_trial_0-Agent-1:	@   12:	We're all 5 changing to Pacha!
INFO:Malasaña_Heavy_trial_0-Agent-1:	@   12:	Pacha now has 81 people inside
INFO:Malasaña_Heavy_trial_0-Agent-2:	@   12:	I'm so drunk. Take me home!
INFO:Malasaña_Heavy_trial_0-Agent-12:	@   12:	I'm so drunk. Take me home!
INFO:Malasaña_Heavy_trial_0-Agent-34:	@   12:	I'm so drunk. Take me home!
INFO:Malasaña_Heavy_trial_0-Agent-47:	@   12:	I'm so drunk. Take me home!
INFO:Malasaña_Heavy_trial_0-Agent-96:	@   12:	I'm so drunk. Take me home!
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@   13:	Kicking out the trash: 2
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@   13:	Kicking out the trash: 12
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@   13:	Kicking out the trash: 34
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@   13:	Kicking out the trash: 47
INFO:Malasaña_Heavy_trial_0-Agent-Police:	@   13:	Kicking out the trash: 96
INFO:Malasaña_Hea

In [6]:
"""config = {
  
    'name': 'pubcrawl_light',
    'num_trials': 3,
    'max_time': 15,

    'network_params': {
        'generator': 'empty_graph',
        'n': 14,
    },
    
    
    
    'network_agents': [
        {
            'agent_type': Patron,
            'description': 'Female Drinker',
            'weight': 5,
            'state': {
                  'prob_drink': 0.3,
                  'prob_change_bar': 0.3,
                  'max_pints': 3,
                  
                
            }
        },
        { 
            'agent_type': Patron,
            'description': 'Male Drinker',
            'weight': 5,
            'state': {
                  'prob_drink': 0.8,
                  'prob_change_bar': 0.3,
                  'max_pints': 5,
                 
            }
        },
    ],
    
    'environment_agents':[
        {'agent_type': Police,
        
        }
    ],
    'environment_class':CityPubs,
    'environment_params':{
       
          'number_of_pubs': 6,
          'pub_capacity': 10,
        
    }

  
  

    
}
"""

"config = {\n  \n    'name': 'pubcrawl_light',\n    'num_trials': 3,\n    'max_time': 15,\n\n    'network_params': {\n        'generator': 'empty_graph',\n        'n': 14,\n    },\n    \n    \n    \n    'network_agents': [\n        {\n            'agent_type': Patron,\n            'description': 'Female Drinker',\n            'weight': 5,\n            'state': {\n                  'prob_drink': 0.3,\n                  'prob_change_bar': 0.3,\n                  'max_pints': 3,\n                  \n                \n            }\n        },\n        { \n            'agent_type': Patron,\n            'description': 'Male Drinker',\n            'weight': 5,\n            'state': {\n                  'prob_drink': 0.8,\n                  'prob_change_bar': 0.3,\n                  'max_pints': 5,\n                 \n            }\n        },\n    ],\n    \n    'environment_agents':[\n        {'agent_type': Police,\n        \n        }\n    ],\n    'environment_class':CityPubs,\n    'environ

In [7]:
"""MAX_TIME = 15

env_params = {'number_of_pubs': 6,
              'pub_capacity': 10,}
sim = soil.simulation.SoilSimulation(
   
                                     network_params= {
                                                'generator': 'empty_graph',
                                                'n': 14,
                                                    },
                                     num_trials=1,
                                     max_time=MAX_TIME,
                                     network_agents =[
                                            {
                                                'agent_type': Patron,
                                                'description': 'Female Drinker',
                                                'weight': 5,
                                                'state': {
                                                      'prob_drink': 0.3,
                                                      'prob_change_bar': 0.3,
                                                      'max_pints': 3,


                                                }
                                            },
                                            { 
                                                'agent_type': Patron,
                                                'description': 'Male Drinker',
                                                'weight': 5,
                                                'state': {
                                                      'prob_drink': 0.8,
                                                      'prob_change_bar': 0.3,
                                                      'max_pints': 5,

                                                }
                                            },
                                        ],
                                        
                                        environment_agents=
                                                {'agent_type': Police,

                                                },
                                            
                                        environment_class = CityPubs,
                                        
                            
                                    environment_params=env_params)
                                                              
                                     #network_params=[{'path': '/home/alberto/Escritorio/Drink/test.gexf',
                                                      
                                      #                'weight': WEIGHT_V,
                                       #              },
                                        #             ],
    
                       
                        

                           
                                     
                                  
                                     
env = sim.run_simulation()[0]"""

"MAX_TIME = 15\n\nenv_params = {'number_of_pubs': 6,\n              'pub_capacity': 10,}\nsim = soil.simulation.SoilSimulation(\n   \n                                     network_params= {\n                                                'generator': 'empty_graph',\n                                                'n': 14,\n                                                    },\n                                     num_trials=1,\n                                     max_time=MAX_TIME,\n                                     network_agents =[\n                                            {\n                                                'agent_type': Patron,\n                                                'description': 'Female Drinker',\n                                                'weight': 5,\n                                                'state': {\n                                                      'prob_drink': 0.3,\n                                                      'pro

In [8]:
!cat pubcrawl.yml

---
name: Malasaña_Heavy
num_trials: 1
max_time: 15
dump: false
network_params:
  # Generate 100 empty nodes. They will be assigned a network agent
  generator: empty_graph
  n: 100
network_agents:
  - agent_type: pubcrawl.Patron
    description: Female Drinker
    state:
      prob_drink: 0.3
      prob_change_bar: 0.3
      max_pints: 3
      gender: female
    weight: 3

  - agent_type: pubcrawl.Patron
    description: Male Drinker
    state:
      prob_drink: 0.8
      prob_change_bar: 0.3
      max_pints: 5
      gender: male
    weight: 7
environment_agents:
  - agent_type: pubcrawl.Police
environment_class: pubcrawl.CityPubs
environment_params:
  altercations: 1
  pubs:
    Space Monkey:
      name: Space Monkey
      open: true
      occupancy: 0
      capacity: 10
      occupancy: 0
      price: 4
      type: pub

    Via Lactea:
      name: Via Lactea
      open: true
      occupancy: 0
      capacity: 30
      occupancy: 0
      price: 5
      type: pub


    Dos de mayo:
  

In [9]:
evodumb = analysis.read_data('soil_output/pubcrawl_light/',  group=True, keys=['id']);

ValueError: No objects to concatenate

In [ ]:
analisis = analysis.

In [ ]:
analisis.values


In [ ]:
from soil import history

In [ ]:
h = history.History('mala.sqlite')

In [ ]:
list(h.to_tuples())